# Budget rule Jupyter notebook for the STI budgets simulator

In [7]:
# simple trick to make sure that we work in the same directory when we are inside a jupyter notebook and when we are running the simulation from the main directory
import os
import pandas as pd
import datetime
cwd = os.getcwd()
if cwd.endswith('jupyter_notebooks'):
    os.chdir('..')
from rules import budget

## Parameters

in order to run the simulations, we need to provide a number of parameters. For the first release, we do not know the parameters in advance. Therefore, the easiest way is to use a dictionary of values.

The list of parameters is the following:

| Name | Description | Optional | Sample value | Default value |
| :---- | :----------- | :------------: | :--- | :--- |
| CF | The 'centre financier' we are working with  | **No** | 1234 | |
| `start_date` | The start date of the simulation | **No** | 2015-01-01 | |
| `end_date` | The end date of the simulation | **No** | 2097-01-01 | |
| `academic_rank` | The current academic rank of the professor in charge of the 'CF' | **No** | 'PATT' | |
| `in_rank_since` | The date when the professor became an `academic_rank` professor | **No** | 2018-01-01 | |
| DOB | Date of Birth of the professor | Yes | 01.01.1970 | 65 years after the promotion as PO |
| PATT_yearly_budget | The PATT yearly budget (if different from the default one - 445'000 CHF / year) | Yes | 445000 | 445000 |
| PA_yearly_budget | The PA yearly budget (if different from the default one - average between the PATT and the PO yearly budget) | Yes | 722500 | Average between the PA and PO budget |
| PO_yearly_budget | The PO yearly budget (if different from the default one - 1'000'000 CHF / year ) | Yes | 1000000 | Yes |



In [23]:
parameters = {}
parameters['CF'] = '1234'
parameters['start_date'] = datetime.datetime(1995,1,1)
parameters['end_date'] = datetime.datetime(2100,1,1)
parameters['academic_rank'] = 'PATT'
parameters['in_rank_since'] = datetime.datetime(2000, 1, 1)
parameters['DOB'] = datetime.datetime(1960, 1, 1)
parameters['PATT_yearly_budget'] = 445000
# parameters['PA_yearly_budget'] = 120000
parameters['PO_yearly_budget'] = 1000000


## boilerplates

In [24]:
df = pd.DataFrame()

## Running the rules

This is were the actual code will calculate the entries of the ledger

In [25]:
milestones, df = budget.main(params = parameters)

## Get an overview of the milestones

In [26]:
print(milestones)

{'patt_promotion': datetime.datetime(2000, 1, 1, 0, 0), 'first_bump_budget_increase_date': Timestamp('2003-01-01 00:00:00'), 'pa_promotion': Timestamp('2006-10-01 00:00:00'), 'po_promotion': Timestamp('2013-07-01 00:00:00'), 'po_step1': Timestamp('2014-07-01 00:00:00'), 'po_step2': Timestamp('2015-07-01 00:00:00'), 'po_step3': Timestamp('2016-07-01 00:00:00'), 'po_full': Timestamp('2017-07-01 00:00:00'), 'retirement': Timestamp('2025-01-01 00:00:00')}


## Create the vizualization of the returned dataframe

In [27]:
from bokeh.plotting import figure, show, output_notebook
output_notebook()
from bokeh.models import ColumnDataSource, CustomJS, NumeralTickFormatter
from bokeh.models.tools import HoverTool
from bokeh.models.widgets import RangeSlider
from bokeh.layouts import column

# Base viz configuration
source = ColumnDataSource(df)
TOOLS = "pan,wheel_zoom,box_zoom,reset,save"
p=figure(x_axis_type='datetime', tools=TOOLS)
p.title.text = "budget simulation"
p.xaxis.axis_label = "date"
p.yaxis.axis_label = "CHF"
p.yaxis.formatter=NumeralTickFormatter(format="0,0")

# Data line
p.line(x='date', y='budget', source=source, line_width=2)

# Hover
hover=HoverTool()
hover.tooltips = [
    ('Date', "@date{%F}"),
    ('Monthly budget', "@budget{'0,0'} CHF"),
    ('Note', '@note'),
]
hover.formatters={
    'date': 'datetime',
    'budget': 'numeral',
}
hover.mode='vline'
p.add_tools(hover)

# Box annotations
from bokeh.models import BoxAnnotation
from bokeh.palettes import OrRd3

no_budget_box = BoxAnnotation(right=milestones['patt_promotion'], fill_alpha=0.1, fill_color='green')
p.add_layout(no_budget_box)
patt_box = BoxAnnotation(left=milestones['patt_promotion'], right=milestones['pa_promotion'], fill_alpha=0.1, fill_color=OrRd3[2])
p.add_layout(patt_box)
pa_box = BoxAnnotation(left=milestones['pa_promotion'], right=milestones['po_promotion'], fill_alpha=0.1, fill_color=OrRd3[1])
p.add_layout(pa_box)
po_box = BoxAnnotation(left=milestones['po_promotion'], right=milestones['retirement'], fill_alpha=0.1, fill_color=OrRd3[0])
p.add_layout(po_box)
retirement_box = BoxAnnotation(left=milestones['retirement'], fill_alpha=0.1, fill_color='green')
p.add_layout(retirement_box)

# Rendering of the viz
layout=column(p, sizing_mode='stretch_width')
show(layout)

Loading BokehJS ...